In [19]:
import psycopg2

def get_cursor():
    # Connect to DB
    conn = psycopg2.connect(
        dbname="vector_db",
        user="user",
        password="123456",
        host="pgvector",
        port=5432
    )
    cur = conn.cursor()
    
    # 1. Enable pgvector extension
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
    return cur, conn

def select(sql_statement):
    cur, conn = get_cursor()
    cur.execute(sql_statement)
    res = cur.fetchall()
    return res

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import psycopg2

import joblib
global vectorizer
vectorizer = joblib.load('vectorizer.joblib')

def get_similar_entries(query):
    q_vec = vectorizer.transform([query]).toarray()[0]
    
    q_vec_str = str(list(q_vec))
    
    # Run similarity search using the <-> operator (Euclidean distance) 
    cur, conn = get_cursor()
    cur.execute("""
        SELECT id, content, embedding <-> %s::vector AS distance
        FROM documents
        ORDER BY distance ASC
        LIMIT 5
    """, (q_vec_str,))

    return cur.fetchall()

query = "Wie oft sollen wir dafür sorgen, dass der Ständer mit neusten Aufgaben befüllt ist?"

results = get_similar_entries(query)

for rank, (doc_id, content, distance) in enumerate(results, 1):
    print(f"\n#{rank} (distance={distance:.4f})\n{content[:300]}...")


#1 (distance=1.0000)
"></p><p></p><p></p>...

#2 (distance=1.0000)
p>...

#3 (distance=1.0000)
p><p></p>...

#4 (distance=1.0093)
<p>Bitte sorgt dafür, dass der Ständer täglich mit den neuesten Ausgaben befüllt ist.</p><p><img src="/geteilte-inhalte/9cea8aa0-558c-4da0-8cf2-2e51d9d2a6e4?showFileContent=1&amp;p=medium"></p><p></p><p></p>...

#5 (distance=1.3180)
<p>Zur Präsentation eines vollen und abwechslungsreichen <strong>Sortimentes</strong>, wie es in jeder unserer Bäckereifiliale zu finden ist, gehört auch, dass die sonst so trübe und leblos erscheinende Thekenablage, welche ständig im Blickfeld des Kunden ist entsprechend genutzt wird.</p><p><img sr...


In [21]:
#import requests
import time
#!pip install scikit-learn llama-cpp-python

def query_llm(model, prompt):
    url = "http://ollama:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
    }

    start_time = time.time()
    response = requests.post(url, json=payload)
    end_time = time.time()

    result = response.json()["response"]
    duration = end_time - start_time

    print(f"Response: {result}")
    print(f"\n⏱️ Time taken: {duration:.2f} seconds")

query_llm("mistral", "Wie oft sollen wir dafür sorgen, dass der Ständer mit neusten Aufgaben befüllt ist?")

Response:  Um einen effektiven und kontinuierlichen Arbeitsfluss zu gewährleisten, sollte der Ständer regelmäßig mit neuen Aufgaben befüllt werden. Je nach Größe und Art der Projekte kann dies täglich, wöchentlich oder monatlich passieren. Es gibt kein allgemeingültiges Regelwerk dafür, wie oft ein Ständer mit neuen Aufgaben befüllt werden soll. Hierbei ist wichtig, sich die Arbeitsbelastung und den Ablauf der Projekte anzupassen.

Es empfele ich folgende Prinzipien:

1. Priorisierung: Verwende prioritätsbasierte Arbeitsspeicherung, damit du die wichtigsten Aufgaben im Ständer bekommst und nicht vergessen kannst.
2. Flexibilität: Sei flexibel und passe deine Prioritäten an die Änderungen in der Arbeit oder den Umständen an.
3. Effizienz: Nutze deinen Ständer effizient, um sich auf wichtige Tätigkeiten zu konzentrieren und Zeit zu sparen.
4. Veränderbarkeit: Verwende einen flexiblen Systemansatz, damit du leicht darauf reagieren kannst, wenn neue Aufgaben auftauchen oder deine Priorität

In [22]:
def ask_with_context(query, model="mistral"):
    results = get_similar_entries(query)

    # Build context from top_k entries
    context = "\n\n".join([f"#{i+1}:\n{content}" for i, (doc_id, content, dist) in enumerate(results)])

    # Build the final prompt
    prompt = f"""Answer the following using the provided context.

Context:
{context}

Question:
{query}

Answer:"""
    print(prompt)
    print()
    print()
    query_llm(model=model, prompt=prompt)

ask_with_context(query)

Answer the following using the provided context.

Context:
#1:
"></p><p></p><p></p>

#2:
p>

#3:
p><p></p>

#4:
<p>Bitte sorgt dafür, dass der Ständer täglich mit den neuesten Ausgaben befüllt ist.</p><p><img src="/geteilte-inhalte/9cea8aa0-558c-4da0-8cf2-2e51d9d2a6e4?showFileContent=1&amp;p=medium"></p><p></p><p></p>

#5:
<p>Zur Präsentation eines vollen und abwechslungsreichen <strong>Sortimentes</strong>, wie es in jeder unserer Bäckereifiliale zu finden ist, gehört auch, dass die sonst so trübe und leblos erscheinende Thekenablage, welche ständig im Blickfeld des Kunden ist entsprechend genutzt wird.</p><p><img sr

Question:
Wie oft sollen wir dafür sorgen, dass der Ständer mit neusten Aufgaben befüllt ist?

Answer:


Response:  Täglich sollte der Ständer mit den neuesten Ausgaben befüllt werden.

⏱️ Time taken: 29.85 seconds
